In [1]:
import matplotlib, os, random
import numpy as np
import imageio.v3 as imageio
from shared.Render import collect_saved_frames, render_TSP
matplotlib.use('Agg')
np.random.seed(42)

#### Helper functions for the ant colony optimization

In [2]:
def evalueate_TSP_path(ids: list, positions: np.ndarray):
    distance = 0
    for idx in range(len(ids) - 1):
        id1, id2 = ids[idx], ids[idx + 1]
        diff_x = positions[id1][0] - positions[id2][0]
        diff_y = positions[id1][1] - positions[id2][1]
        distance += np.sqrt(diff_x**2 + diff_y**2)
    id1, id2 = ids[-1], ids[0]
    diff_x = positions[id1][0] - positions[id2][0]
    diff_y = positions[id1][1] - positions[id2][1]
    distance += np.sqrt(diff_x**2 + diff_y**2)
    return distance

In [3]:
def compute_distance_matrix(positions: np.ndarray):
    diff = positions[:, np.newaxis, :] - positions[np.newaxis, :, :]
    dist = np.sqrt(np.sum(diff**2, axis=2))
    np.fill_diagonal(dist, np.inf)
    return dist

In [4]:
def choose_next_city(current, unvisited, pheromone, heuristic, alpha, beta):
    candidates = list(unvisited)
    tau = pheromone[current][candidates]
    eta = heuristic[current][candidates]
    weights = (tau ** alpha) * (eta ** beta)
    total = np.sum(weights)
    if total == 0:
        return random.choice(candidates)
    probabilities = weights / total
    return np.random.choice(candidates, p=probabilities)

In [5]:
def update_pheromones(pheromone, paths, distances, rho, q):
    pheromone *= (1 - rho)
    pheromone = np.maximum(pheromone, 1e-12)
    for path, distance in zip(paths, distances):
        if distance == 0:
            continue
        deposit = q / distance
        for idx in range(len(path)):
            a = path[idx]
            b = path[(idx + 1) % len(path)]
            pheromone[a, b] += deposit
            pheromone[b, a] += deposit
    np.fill_diagonal(pheromone, 0.0)
    return pheromone

#### The ant colony optimization algorithm for TSP problem

In [6]:
def ant_colony_optimization(
        positions: np.ndarray,
        num_ants: int = 20,
        num_iter: int = 100,
        alpha: float = 1.0,
        beta: float = 5.0,
        rho: float = 0.5,
        q: float = 100.0):
    num_cities = len(positions)
    distances = compute_distance_matrix(positions)
    heuristic = 1.0 / distances
    heuristic[~np.isfinite(heuristic)] = 0
    pheromone = np.ones((num_cities, num_cities))
    np.fill_diagonal(pheromone, 0.0)
    best_distance = float("inf")
    for iter in range(num_iter):
        paths = []
        values = []
        for ant_idx in range(num_ants):
            start = random.randrange(num_cities)
            path = [start]
            unvisited = set(range(num_cities))
            unvisited.remove(start)
            while unvisited:
                current = path[-1]
                next_city = choose_next_city(current, unvisited, pheromone, heuristic, alpha, beta)
                path.append(next_city)
                unvisited.remove(next_city)
            path = np.array(path)
            path_value = evalueate_TSP_path(path, positions)
            paths.append(path)
            values.append(path_value)
            improved = False
            if path_value < best_distance:
                best_distance = path_value
                improved = True
            yield iter, ant_idx, path, path_value, improved, best_distance
        pheromone = update_pheromones(pheromone, paths, values, rho, q)

#### Generate random positions on 2D plane

In [7]:
num_positions = 15
positions_range = (-20, 20)
positions = np.random.uniform(positions_range[0], positions_range[1], size=(num_positions, 2))

#### Run the ant colony optimization and collect frames

In [8]:
gif_folder_path = "gifs/aco"
os.makedirs(gif_folder_path, exist_ok=True)

In [9]:
for idx, (iteration, ant_idx, tour, tour_distance, improved, best_distance) in enumerate(ant_colony_optimization(positions)):
    if improved:
        text = f"Iteration: {iteration} ant {ant_idx} improved best, distance: {tour_distance:.2f}"
    else:
        text = f"Iteration: {iteration} ant {ant_idx}, distance: {tour_distance:.2f} (best: {best_distance:.2f})"
    frame = render_TSP(tour, positions, text)
    imageio.imwrite(os.path.join(gif_folder_path, f"aco{idx:08d}.png"), frame)
collect_saved_frames(gif_folder_path, f"AntColony.gif", fps=2)

#### Visualize the ant colony optimization on the TSP problem

![AntColony](gifs/aco/AntColony.gif)